### 1.Tool cration

In [1]:
from langchain_core.tools import tool,InjectedToolArg
import requests
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_core.messages import HumanMessage
from typing import Annotated

e:\1.3.5.7.9\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@tool
def get_converstation_factor(base_currency:str, target_currency:str) -> float:
    """This function fetches the currency conversion factor between a given base currency and a target currency"""
    
    url=f"https://v6.exchangerate-api.com/v6/97899f18859aee6a0dd3d656/pair/{base_currency}/{target_currency}"
    
    response=requests.get(url)
    
    return response.json()

In [3]:
get_converstation_factor.invoke({'base_currency':"USD",'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1771200002,
 'time_last_update_utc': 'Mon, 16 Feb 2026 00:00:02 +0000',
 'time_next_update_unix': 1771286402,
 'time_next_update_utc': 'Tue, 17 Feb 2026 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 90.6677}

In [4]:
@tool
def convert(base_currency : int ,conversion_rate : Annotated [ float , InjectedToolArg ] ) -> float:
    """given a currency rate this function calculates the target currency value from a given base currency value"""
    
    return base_currency*conversion_rate

In [5]:
convert.invoke({'base_currency':10,'conversion_rate':90.6677})

906.6769999999999

### 2.binding tool

In [6]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq

load_dotenv()

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct-0905",  # ✅ spelling fixed
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.3,
)

response = llm.invoke("Hi")
print(response.content)


Hi! How can I help you today?


In [7]:
llm_with_tool=llm.bind_tools([get_converstation_factor,convert])

In [8]:
messages = [
    HumanMessage(
        content="The exchange rate for USD to INR is 83.5. "
                "Use the get_converstation_factor tool to verify this, "
                "and simultaneously use the convert tool to change 10 USD to INR."
    )
]

### so here we learn about promting

In [9]:
"""“By structuring prompts to assign explicit responsibilities to tools, such as verification vs computation, reducing model autonomy.”"""

'“By structuring prompts to assign explicit responsibilities to tools, such as verification vs computation, reducing model autonomy.”'

In [10]:
"""messages=[HumanMessage('what is conversion factor between USD and INR , and based on that can you conver 10 usd to inr')]"""

"messages=[HumanMessage('what is conversion factor between USD and INR , and based on that can you conver 10 usd to inr')]"

In [11]:
"""“By structuring prompts to assign explicit responsibilities to tools, such as verification vs computation, reducing model autonomy.”"""

'“By structuring prompts to assign explicit responsibilities to tools, such as verification vs computation, reducing model autonomy.”'

In [12]:
"""messages = [
    HumanMessage(
        content="The exchange rate for USD to INR is what?. "
                "Use the get_converstation_factor tool to verify this, "
                "and simultaneously use the convert tool to change 10 USD to INR."
    )
]"""
# in gemini and open ai there is promt technique is very differnce

'messages = [\n    HumanMessage(\n        content="The exchange rate for USD to INR is what?. "\n                "Use the get_converstation_factor tool to verify this, "\n                "and simultaneously use the convert tool to change 10 USD to INR."\n    )\n]'

In [13]:
# messages=[HumanMessage('The exchange rate for USD to INR is something,Use the get_converstation_factor tool to verify this,and simultaneously use the convert tool to change 10 USD to INR')]

In [14]:
ai_message=llm_with_tool.invoke(messages)

In [15]:
ai_message

AIMessage(content="I'll help you verify the exchange rate and convert 10 USD to INR using the available tools.", additional_kwargs={'tool_calls': [{'id': 'functions.get_converstation_factor:0', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_converstation_factor'}, 'type': 'function'}, {'id': 'functions.convert:1', 'function': {'arguments': '{"base_currency":10}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 226, 'total_tokens': 288, 'completion_time': 0.134076513, 'completion_tokens_details': None, 'prompt_time': 0.023705923, 'prompt_tokens_details': None, 'queue_time': 0.275407137, 'total_time': 0.157782436}, 'model_name': 'moonshotai/kimi-k2-instruct-0905', 'system_fingerprint': 'fp_241bc7119c', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--019c654b-6b17-7f82-9bf5-6c50a67e166e-0', tool_calls=[{'name': 'get_converstation_factor

In [ ]:
ai_message.tool_calls

[{'name': 'get_converstation_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'functions.get_converstation_factor:0',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency': 10},
  'id': 'functions.convert:1',
  'type': 'tool_call'}]

### injected tool argument